In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Chat 모델 및 프롬프트 설정
from langchain.chat_models import ChatOpenAI
from langchain.prompts.example_selector import LengthBasedExampleSelector # 예제 허용 양을 지키며 LLM에게 전달하게 됨  
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector    


# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature= 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()] 
)   

# Few-Shot Learning을 위한 예제 데이터
# AI에게 원하는 답변 형식을 학습시키기 위한 예시들
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

# 1. 예제 하나의 형식 정의
example_prompt = PromptTemplate.from_template(
    "Human: {question} \nAI: {answer}"
)

# 2. 길이 기반으로 예제 선택
# 프롬프트 길이를 고려해서 예제를 선택
# max_length를 초과하지 않도록 예제 개수를 자동 조절
example_selector = LengthBasedExampleSelector(
    examples = examples, # 전체 예제 목록
    example_prompt = example_prompt, # 1번에서 만든 예제 형식
    max_length = 180, # 포맷된 예제들의 최대 허용 문자 수
)
# -> 질문이 길면 예제를 적게, 질문이 짧으면 예제를 많이 포함


# 3. FewShotTemplate 구성
# examples 대신 example_selector 사용
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt, # 1번에서 만든 틀 사용
    example_selector = example_selector, # examples 직접 전달 대신 selector 사용
    # examples를 직접 사용하면 → 모든 예제를 항상 포함
    # example_selector를 사용하면 → 길이에 따라 예제 개수 자동 조절
    suffix = "Human: What do you know about {country}?", # 실제 질문
    input_variables = ["country"] # suffix 에서 사용할 변수
)

# 프롬프트 포맷팅 테스트
prompt.format(country = "Brazil")

'Human: What do you know about France? \nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy? \nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'